In [1]:
from ultralytics import YOLO
import os
import cv2
import shutil
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
import os
import shutil
import cv2
import numpy as np

# Укажите новые пути
IMAGES_TRAIN_DIR = r"C:\Users\user\Desktop\nornik\split_orig_aug_data\open_img\train"
IMAGES_TEST_DIR = r"C:\Users\user\Desktop\nornik\split_orig_aug_data\open_img\test"
MASKS_TRAIN_DIR = r"C:\Users\user\Desktop\nornik\split_orig_aug_data\open_msk\train"
MASKS_TEST_DIR = r"C:\Users\user\Desktop\nornik\split_orig_aug_data\open_msk\test"
OUTPUT_DIR = r'C:\Users\user\Desktop\nornik\split_orig_aug_data_yolo'

# Создание структуры папок
os.makedirs(os.path.join(OUTPUT_DIR, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/val'), exist_ok=True)

def convert_mask_to_yolo(mask_path, output_subdir):
    """Конвертирует маску в формат YOLO."""
    image = cv2.imread(mask_path)
    if image is None:
        print(f"Не удалось открыть изображение: {mask_path}")
        return

    height, width = image.shape[:2]
    black_mask = cv2.inRange(image, (0, 0, 0), (50, 50, 50))
    new_image = np.ones_like(image) * 255
    new_image[black_mask > 0] = [0, 0, 0]
    gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    output_file_name = os.path.splitext(os.path.basename(mask_path))[0] + '.txt'
    output_file_path = os.path.join(OUTPUT_DIR, output_subdir, output_file_name)

    with open(output_file_path, 'w') as f:
        for contour in contours:
            contour_points = contour.reshape(-1, 2)
            normalized_points = [(x / width, y / height) for x, y in contour_points]
            points_str = ' '.join(f"{x:.3f} {y:.3f}" for x, y in normalized_points)
            f.write(f"0 {points_str}\n")

def process_data(images_dir, masks_dir, output_images_subdir, output_labels_subdir):
    """Копирует изображения и конвертирует маски."""
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]
    for img in image_files:
        # Копирование изображения
        shutil.copy(os.path.join(images_dir, img), os.path.join(OUTPUT_DIR, output_images_subdir, img))
        # Конвертация маски
        mask_name = img.replace('.jpg', '.png')
        mask_path = os.path.join(masks_dir, mask_name)
        if os.path.exists(mask_path):
            convert_mask_to_yolo(mask_path, output_labels_subdir)
        else:
            print(f"Маска отсутствует для изображения: {img}")

# Обработка обучающего и тестового набора
process_data(IMAGES_TRAIN_DIR, MASKS_TRAIN_DIR, 'images/train', 'labels/train')
process_data(IMAGES_TEST_DIR, MASKS_TEST_DIR, 'images/val', 'labels/val')

# Создание файла data.yaml
data_yaml_content = f"""
train: {os.path.join(OUTPUT_DIR, 'images/train')}
val: {os.path.join(OUTPUT_DIR, 'images/val')}

nc: 1  # Обновите количество классов (1 для загрязнения)
names: ['contaminated']  # Обновите названия классов
"""

yaml_path = os.path.join(OUTPUT_DIR, 'data.yaml')
with open(yaml_path, 'w') as f:
    f.write(data_yaml_content)

print(f"Датасет успешно обработан и сохранен в: {OUTPUT_DIR}")

In [ ]:
model = YOLO("yolo11n-seg.pt")

  0%|          | 0.00/19.7M [00:00<?, ?B/s]

In [ ]:
import torch


torch.set_num_threads(1)

train_results = model.train(
    data=r"C:\Users\user\Desktop\nornik\split_orig_aug_data_yolo\data.yaml",  # path to dataset YAML
    epochs=1000,  # number of training epochs
    imgsz=640,  # training image size
    device=0,  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    patience=100,
    batch=16,
)

New https://pypi.org/project/ultralytics/8.3.47 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.38  Python-3.10.7 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24564MiB)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=segment, mode=train, model=yolo11s-seg.pt, data=C:\Users\user\Desktop\nornik\split_data2_yolo\data.yaml, epochs=1000, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=F

train: Scanning C:\Users\user\Desktop\nornik\split_data2_yolo\labels\train.cache... 250 images, 163 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\user\Desktop\nornik\split_data2_yolo\labels\val.cache... 250 images, 163 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]


Plotting labels to runs\segment\train12\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.0005), 100 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\segment\train12
Starting training for 1000 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

   254/1000      5.07G     0.7297      1.272     0.6357      1.059         15        640: 100%|██████████| 16/16 [00:02<00:00,  6.91it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  6.64it/s]

                   all        250        202       0.94      0.936      0.965      0.799      0.935      0.931      0.955      0.698



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


   255/1000      5.05G     0.7659      1.345     0.6619      1.077         17        640: 100%|██████████| 16/16 [00:02<00:00,  6.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  6.72it/s]

                   all        250        202      0.913      0.936      0.966      0.801      0.905      0.926      0.956      0.708
